In [466]:
#### Code written by Pankaj and verified the logic with the blog - 
# https://towardsdatascience.com/linear-regression-from-scratch-with-numpy-implementation-finally-8e617d8e274c
############## Steps to do 
# 1. Generate a dataset ( n rows; m features; 1 target-continuous)
# 2. Write Cost function 
# 3. Write gradient descent function
# 4. Write Prediction function 
# 5. Write train function

# For KNN in numpy: https://towardsdatascience.com/k-nearest-neighbors-classification-from-scratch-with-numpy-cb222ecfeac1
# For Kmeans in numpy: https://flothesof.github.io/k-means-numpy.html

In [2]:
import numpy as np
import sys, os, csv, random
from sklearn import metrics
from collections import Counter

In [178]:
# 1. Data Generation
data = []
for _ in range(1000):
    a = np.random.rand(5)*100 +5
    data.append(a)
data = np.vstack((data))

In [179]:
# Linear regression class
class linear_reg:
    def __init__(self):
        self.X1 = None
        self.y = None
        self.W = None
        self.num_epochs = 10
        self.lr = 0.01
        
    def gradient(self):
        n,m = self.X1.shape
        diff = (self.y - self.__predict(self.X1)).T
        res = np.matmul(diff, self.X1)
        C = (-2.0/n)
        return C*res.T
    
    def __train(self):
        # Initiliaze weights
        n,m = self.X1.shape
        self.W = np.random.rand(m)
        #self.W = np.array([np.random.rand() for _ in range(m)])
        loss = []
        for epoch in range(self.num_epochs):
            self.W = self.W - self.lr*self.gradient()
            curr_loss = (self.loss_func(self.y, self.__predict(self.X1)))
            loss.append(curr_loss)
            print (curr_loss)
        
    def train(self,X,y):
        self.X1 = np.hstack((np.ones((X.shape[0],1)), X))
        self.y = y
        self.__train() # Begin training
        
        
    def __predict(self,X_eval):
        #return np.dot(W, x_vector)
        #return np.matmul(X_eval, self.W.T)
        return X_eval@self.W.T
    
    def predict(self, X_temp):
        #print (X_temp.shape)
        temp  = np.hstack((np.ones((X_temp.shape[0],1)), X_temp))
        #print (temp.shape)
        return self.__predict(temp)
    
    def loss_func(self,y, y_hat):
        return np.mean((y - y_hat)**2)

In [ ]:
X, y = data[:,:-1], data[:,-1]
model = linear_reg()
model.train(X,y)
model.predict(X)

In [189]:
# Logistic regression class
class logistic_reg:
    def __init__(self):
        self.X1 = None
        self.y = None
        self.W = None
        self.num_epochs = 10
        self.lr = 0.01
        
    def gradient(self):
        n,m = self.X1.shape
        diff = (self.y - self.__predict(self.X1)).T
        res = np.matmul(diff, self.X1)
        C = (-1.0/n)
        return C*res.T
    
    def __train(self):
        # Initiliaze weights
        n,m = self.X1.shape
        self.W = np.random.rand(m)
        #self.W = np.array([np.random.rand() for _ in range(m)])
        loss = []
        for epoch in range(self.num_epochs):
            self.W = self.W - self.lr*self.gradient()
            curr_loss = (self.loss_func(self.y, self.__predict(self.X1)))
            loss.append(curr_loss)
            print (curr_loss)
        
    def train(self,X,y):
        X = (X -np.mean(X))/X.std()
        self.X1 = np.hstack((np.ones((X.shape[0],1)), X))
        self.y = y
        self.__train() # Begin training
        
        
    def __predict(self,X_eval):
        #return np.dot(W, x_vector)
        #return np.matmul(X_eval, self.W.T)
        return self.sigmoid(X_eval@self.W.T)
    
    def sigmoid(self,z):
        return 1.0/(1+np.exp(-z))
    
    def predict(self, X_temp):
        X_temp = (X_temp -np.mean(X_temp))/X_temp.std()
        temp  = np.hstack((np.ones((X_temp.shape[0],1)), X_temp))
        #print (temp.shape)
        return self.__predict(temp)
    
    def loss_func(self,y, y_hat):
        cost = -1*(y*np.log(y_hat) + (1-y)*np.log(1-y_hat))
        return np.mean(cost)

In [ ]:
X, y = data[:,:-1], np.random.choice([0,1], data.shape[0])
model = logistic_reg()
model.train(X,y)
model.predict(X)

In [7]:
# Compute AUC; mostly written by me with some help from StackOverflow
def give_tpr_fpr(y, y_pred, cutoff):
    y_hat = np.where(y_pred >= cutoff, 1, 0)
    TP = np.sum((y == 1) & (y_hat == 1))
    FP = np.sum((y == 0) & (y_hat == 1))
    FN = np.sum((y == 1) & (y_hat == 0))
    TN = np.sum((y == 0) & (y_hat == 0))
    #print (TP, FP, FN, TN, y_hat)
    
    tpr = TP/(TP+FN)
    fpr = FP/(FP+TN)
    return fpr, tpr  

def compute_auc(y, y_pred):
    thresholds = np.sort(y_pred) # thresholds in ascending order
    print(thresholds)
    #np.hstack((np.array([0.]), y_pred,np.array([1.]))) # appended 0 and 1 in the end 
    points = []
    for cutoff in thresholds:
        point = give_tpr_fpr(y, y_pred, cutoff)
        points.append(point)
    
    points = points[::-1]
    print(points)
    auc = 0
    for i in range(len(points) -1):
        dx = points[i+1][0] - points[i][0]
        y_avg = (points[i+1][1] + points[i][1])/2
        auc += dx*y_avg
    return auc

y_pred = np.array([0.7, 0.1, 0.5, 0.8, 0.2, 0.7, 0.3])
y = np.array([1,0,0,0,1,1,0])
print (compute_auc(y, y_pred))
### for verification
fpr, tpr, thresholds = metrics.roc_curve(y, y_pred)
print (metrics.auc(fpr, tpr))

[0.1 0.2 0.3 0.5 0.7 0.7 0.8]
[(0.25, 0.0), (0.25, 0.6666666666666666), (0.25, 0.6666666666666666), (0.5, 0.6666666666666666), (0.75, 0.6666666666666666), (0.75, 1.0), (1.0, 1.0)]
0.5833333333333333
0.5833333333333333


In [6]:
thresholds

array([1.8, 0.8, 0.7, 0.3, 0.2, 0.1])

In [424]:
# Written purely by me after some rial and error 
def compute_percentile(a, perc):
    #perc = 100
    a_sorted = np.sort(a)

    end = len(a) - 1
    start = 0 
    index = start + ( end - start)*perc/100
    #print (index)
    int_index = int(index)
    rem_index = index - int_index
    next_index = int_index + 1
    if next_index == len(a): next_index -= 1 # Just a corner case when perc == 100; else not required
    val = a_sorted[int_index] + rem_index*(a_sorted[next_index] - a_sorted[int_index])
    return (index, val)
 
# Code for percentile
a = np.random.rand(180)
#a = [10,20,30,40]
print (compute_percentile(a, 90))

#compared with actual -- its matching
print (np.percentile(a, 90))

(161.1, 0.9132139995198986)
0.9132139995198986


In [459]:
## Generate rand7() ; given rand5()
# Source - https://leetcode.com/problems/implement-rand10-using-rand7/solution/
# Source - https://stackoverflow.com/questions/137783/expand-a-random-range-from-1-5-to-1-7
def rand5():
    return np.random.choice(5)+1

def rand7():
    while(True):
        row = rand5()
        col = rand5()
        count = (row -1)*5 + col 
        if count <= 21:    
            return count%7+1
Counter([rand7() for _ in range(100000)])

# Expected number of calls to rand5() to generate one rand7()
# prob of success = 21/25; From geometric distribution: Expected value = 25/21 ; calls = 2*25/21 = 2.38
# must watch approach -2 in leetcode ; reduces the wastage to next level

Counter({2: 14258, 5: 14248, 3: 14211, 6: 14226, 4: 14381, 1: 14412, 7: 14264})

In [460]:
d =  [10,20,30]

In [462]:
list(enumerate(d))

[(0, 10), (1, 20), (2, 30)]